In [1]:
import csv
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
import tqdm
import json
from collections import defaultdict
import datetime

In [2]:
path = '../news data/comments/daily_comments/'
daily_comments = os.listdir(path)
len(daily_comments)

1308

In [ ]:
# 성지순례를 위한 데이터셋 만드는 작업 돌입.
# 이해를 잘못했네... daily comments 파일 ==> 각 날짜에 만들어진 기사에 대한 모든 댓글 포함! ==> 각 comments 안에서 timeline을 찾아야 하는 거임.

def created_time (createdAt):
    date = createdAt[:10]
    return ''.join(date.split('-'))

def criteria (timeline, threshold_freq = 4, threshold_sum = 30, threshold_date = 183):
    # 해당 조건을 만족해야만 성지로서의 자격을 1차로 얻을 수 있다고 볼 수 있음.
    
    # 1. timeline의 길이가 treshold_freq 보다 길어야 함.
    if len(timeline) < threshold_freq:
        return False
    
    # 2. 해당 기사의 총 댓글 수가 threshold_sum 보다 많아야 함.
    total = sum(list(timeline.values()))
    if total < threshold_sum:
        return False
    
    # 3. 가장 최근 댓글과 발행일 사이의 간격이 threshold_date 보다 길어야 함.
    published_date = datetime.datetime.strptime(list(timeline.keys())[0], '%Y%m%d')
    last_comment_date = datetime.datetime.strptime(list(timeline.keys())[-1], '%Y%m%d')
    term = (last_comment_date - published_date).days 
    if term < threshold_date:
        return False
    
    return True

    


comment_path = '../news data/comments/daily_comments/'
article_path = '../news data/articles/all_daily_articles_naver/'

for day in tqdm.tqdm(daily_comments[517:]):
    pilgrim_info = {}
    with open(comment_path + day) as cfile:
        comment_data = json.load(cfile)
    
    with open(article_path + day) as afile:
        article_data = json.load(afile)

    date = day[:8]
    candidate_counter = 0

    for aid, comments in comment_data.items():
        comment_timeline = defaultdict(int)
        for comment in comments:
            timestamp = created_time(comment['createdAt'])
            comment_timeline[timestamp] += 1
            
        if criteria(comment_timeline):
            pilgrim_info[aid] = {}
            pilgrim_info[aid]['url'] = article_data[aid]['url']
            pilgrim_info[aid]['office'] = article_data[aid]['office']
            pilgrim_info[aid]['title'] = article_data[aid]['title']
            pilgrim_info[aid]['published_date'] = article_data[aid]['timestamp']
            pilgrim_info[aid]['timeline'] = comment_timeline
            candidate_counter += 1
            
#     print(f'{date}: 1ST PILGRIM CANDIDATES: {candidate_counter}\tTOTAL COMMENTS: {len(comment_data)}')
            
    pilgrim_info = sorted(pilgrim_info.items(), key = lambda x: len(x[1]['timeline']), reverse = True)
    with open('../pilgrim data/candidates/' + day, 'w') as f:
        json.dump(pilgrim_info, f)

100%|█████████▉| 790/791 [1:06:02<00:05,  5.16s/it]

In [2]:
pilgrim_info

NameError: name 'pilgrim_info' is not defined